In [1]:
import os, io, shutil
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm

import numpy as np
from glob import glob
from tensorboardX import SummaryWriter
from dataloader.handhygiene import HandHygiene
from openpose_transforms import MultiScaleTorsoRandomCrop
from spatial_transforms import Compose
from spatial_transforms import Normalize
from spatial_transforms import Scale
from spatial_transforms import CenterCrop
from spatial_transforms import RandomHorizontalFlip
from spatial_transforms import RandomAffine
from spatial_transforms import RandomRotation
from spatial_transforms import ColorJitter
from spatial_transforms import ToTensor #ExtractSkinColor
from temporal_transforms import TemporalRandomChoice
from temporal_transforms import TemporalRandomCrop
from temporal_transforms import LoopPadding, MirrorPadding, MirrorLoopPadding

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
model_name = 'i3d'
batch_size = 16
clip_length_in_frames = 16
clip_length_aug = clip_length_in_frames/2
num_classes = 1

#torch.manual_seed(100)
data_name = 'anesthesia'
dataset_path = os.path.join(os.getcwd(), 'data')

In [3]:
sample_size = 224
mean=[110.63666788, 103.16065604, 96.29023126]
std=[38.7568578, 37.88248729, 40.02898126]
#mean=[128, 128, 128]
#std=[128, 128, 128]

scales=np.linspace(1, 1.75, num=1e3)
center=((1+1.75)/2)
openpose_transform = {
    'train':MultiScaleTorsoRandomCrop(scales, sample_size),
    'val':MultiScaleTorsoRandomCrop(np.linspace(center, center, num=1), sample_size, centercrop=True)
}
spatial_transform = {
    'train': Compose([Scale(sample_size),
                      CenterCrop(sample_size),
                      RandomHorizontalFlip(),
                      ColorJitter(brightness=0.1),
                      RandomAffine(5),
                      RandomRotation(2.5),
                      ToTensor(1), 
                      Normalize(mean, std)]),
    'val': Compose([Scale(sample_size), 
                    CenterCrop(sample_size), 
                    ToTensor(1), 
                    Normalize(mean, std)])}
temporal_transform = {
    'train':Compose([ #TemporalRandomCrop(clip_length_aug),
                TemporalRandomChoice([
                    LoopPadding(clip_length_in_frames),
                    MirrorPadding(clip_length_in_frames),
                    MirrorLoopPadding(clip_length_in_frames)])]),
    'val': LoopPadding(clip_length_in_frames)}

In [ ]:
dataset = {
    'train':HandHygiene(dataset_path, split='train', 
                        clip_length_in_frames=clip_length_in_frames, 
                        spatial_transform=spatial_transform['train'],
                        openpose_transform=openpose_transform['train'],
                        temporal_transform=temporal_transform['train']
                       ),
    'val':HandHygiene(dataset_path, split='val',
                        clip_length_in_frames=clip_length_in_frames, 
                        spatial_transform=spatial_transform['val'],
                        openpose_transform=openpose_transform['val'],
                        temporal_transform=temporal_transform['val']
                     ),
    'test':HandHygiene(dataset_path, split='test', 
                        clip_length_in_frames=clip_length_in_frames, 
                        spatial_transform=spatial_transform['val'],
                        openpose_transform=openpose_transform['val'],
                        temporal_transform=temporal_transform['val']
                      )}

dataloaders = {
    'train': DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=16),
    'val': DataLoader(dataset['val'], batch_size=batch_size, shuffle=False, num_workers=16)}

In [ ]:
for phase in ['train', 'val', 'test']:
    dataset[phase].__ref__()

# Training

In [ ]:
from train import get_models
from train import train
from torchsummary import summary

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def show_dataset(item): # item: C x D x 244 x 244
    rgbs = item[0].transpose(0, 1)
    flows = item[1].transpose(0, 1)
    n = len(rgbs)
    shape = np.asarray(flows[0]).shape
    rgb = np.hstack((np.asarray(rgb).transpose(1, 2, 0)+1)/2 for rgb in rgbs)
    #gray = np.hstack((np.squeeze(np.asarray(gray))+1)/2 for gray in rgbs)
    tmp = np.zeros((shape[1], shape[2], 1))
    flow = np.hstack((np.dstack((np.asarray(flow).transpose(1, 2, 0), tmp))+1)/2 for flow in flows)

    img = np.vstack((rgb, flow))
    plt.figure(figsize=(50, 10))
    plt.imshow(img)
    #plt.imshow(gray)
    plt.axis('off')
    plt.show()
    print(item[2])

i=400
phase='train'
show_dataset(dataset[phase].__getitem__(i))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
i3d_rgb, i3d_flow = get_models(num_classes, True, 170, load_pt_weights=True) # unfreeze last mix 170, 152
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    i3d_rgb = nn.DataParallel(i3d_rgb)
    i3d_flow = nn.DataParallel(i3d_flow)
i3d_rgb.to(device)
i3d_flow.to(device)

criterion = F.binary_cross_entropy
optims={'rgb':None, 'flow':None}
schedulers = {'rgb':None, 'flow':None}
feature_extract=True

In [ ]:
def trainable_params(model, mode='rgb'):
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    optims[mode] = optim.SGD(model.parameters(), lr=1e-6, momentum=0.9, weight_decay=1e-7)

trainable_params(i3d_rgb, 'rgb')
trainable_params(i3d_flow, 'flow')
    
schedulers['rgb'] = MultiStepLR(optims['rgb'], milestones=[10], gamma=0.1)
schedulers['flow'] = MultiStepLR(optims['flow'], milestones=[10], gamma=0.1)

In [ ]:
#summary(i3d_rgb, (3, 16, 224, 224))

In [ ]:
train((i3d_rgb, i3d_flow), dataloaders, optims, criterion, schedulers, device, num_epochs=200)

# 5-Fold Validation

In [ ]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split

for label in ['clean', 'notclean']:
    data_all = glob('./data/images/{}/*'.format(label))
    data_train, data_test = train_test_split(data_all, test_size=0.1, random_state=42)
    data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)
    data = {'train': data_train, 'val': data_val, 'test':data_test}
    for phase in ['train', 'val', 'test']:
        for path in data[phase]:
            dst= './data/images/{}/{}'.format(phase, label)
            !mv $path $dst

# EDA

In [ ]:
from glob import glob
clean = {'train':[], 'val':[], 'test':[]}
notclean = {'train':[], 'val':[], 'test':[]}
clean_num = {'train':[], 'val':[], 'test':[]}
notclean_num = {'train':[], 'val':[], 'test':[]}

for phase in ['train', 'val', 'test']:
    paths = [path for path in dataset[phase].samples[0] if len(os.path.basename(path).split('_')) == 3]
    clean[phase] = [path for path in paths if '/clean/' in path]
    notclean[phase] = [path for path in paths if '/notclean/' in path]
for phase in ['train', 'val', 'test']:
    for f in clean[phase]: 
        num = len(glob(os.path.join(f, '*.jpg')))
        clean_num[phase].append(num)
    for f in notclean[phase]: 
        num = len(glob(os.path.join(f, '*.jpg')))
        notclean_num[phase].append(num)

print('\tclips', '\timages', '\tmin', '\tmax')
for key, value in clean_num.items():
    print(key, '\t%d\t%d\t%d\t%d' % (len(value), sum(value), min(value), max(value)))
print('\tclips', '\timages', '\tmin', '\tmax')
for key, value in notclean_num.items():
    print(key, '\t%d\t%d\t%d\t%d' % (len(value), sum(value), min(value), max(value)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def draw_result(lst_iter, lst_loss, lst_acc, title):
    plt.plot(lst_iter, lst_loss, '-b', label='loss')
    plt.plot(lst_iter, lst_acc, '-r', label='accuracy')

    plt.xlabel("n iteration")
    plt.legend(loc='upper left')
    plt.title(title)

    # save image
    plt.savefig(title+".png")  # should before show method

    # show
    plt.show()


def test_draw():
    # iteration num
    lst_iter = range(100)

    # loss of iteration
    lst_loss = [0.01 * i + 0.01 * i ** 2 for i in xrange(100)]
    # lst_loss = np.random.randn(1, 100).reshape((100, ))

    # accuracy of iteration
    lst_acc = [0.01 * i - 0.01 * i ** 2 for i in xrange(100)]
    # lst_acc = np.random.randn(1, 100).reshape((100, ))
    draw_result(lst_iter, lst_loss, lst_acc, "sgd_method")


if __name__ == '__main__':
    test_draw()